In [ ]:
import pandas as pd
fetal=pd.read_csv('../input/fetal-health-classification/fetal_health.csv')
fetal.head()

In [ ]:
fetal.info()

In [ ]:
fetal.describe()

In [ ]:
round(fetal.fetal_health.value_counts()*100/len(fetal),2)

In [ ]:
round(fetal.severe_decelerations.value_counts()*100/len(fetal),2)

In [ ]:
round(fetal.histogram_tendency.value_counts()*100/len(fetal),2)

In [ ]:
fetal.nunique().sort_values(ascending=True)

In [ ]:
fetal.skew().sort_values(ascending=True)

In [ ]:
fetal=fetal.drop_duplicates()
fetal.shape

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns
for i in fetal.columns.tolist():
    plt.figsize=(10,5)
    sns.boxplot(y=fetal[i])
    plt.show()

In [ ]:
!pip install pycaret

In [ ]:
data = fetal.sample(frac=0.95, random_state=42)
data_unseen = fetal.drop(data.index)
data.reset_index(inplace=True, drop=True)
data_unseen.reset_index(inplace=True, drop=True)
print('Data for Modeling: ' + str(data.shape))

In [ ]:
print('Unseen Data For Predictions: ' + str(data_unseen.shape))

In [ ]:
from pycaret.classification import *

In [ ]:
fetal_helath= setup(data = data, target = 'fetal_health', session_id=42,
                  normalize = True, 
                  transformation = True, 
                  log_experiment = True,
                  handle_unknown_categorical = True, 
                  unknown_categorical_method = 'most_frequent',
                  remove_multicollinearity = True, #drop one of the two features that are highly correlated with each other
                  ignore_low_variance = True,#all categorical features with statistically insignificant variances are removed from the dataset.
                  combine_rare_levels = True,# all levels in categorical features below the threshold defined in rare_level_threshold param are combined together as a single level
                  numeric_imputation='median',
                    normalize_method='robust',    
                    numeric_features=['baseline value', 'accelerations', 'fetal_movement',
       'uterine_contractions', 'light_decelerations',
       'prolongued_decelerations', 'abnormal_short_term_variability',
       'mean_value_of_short_term_variability',
       'percentage_of_time_with_abnormal_long_term_variability',
       'mean_value_of_long_term_variability', 'histogram_width',
       'histogram_min', 'histogram_max', 'histogram_number_of_peaks',
       'histogram_number_of_zeroes', 'histogram_mode', 'histogram_mean',
       'histogram_median', 'histogram_variance'],
                    categorical_features=['severe_decelerations','histogram_tendency'],
                    
                    
           fix_imbalance = True,
            train_size = 0.8,
          )



In [ ]:
lightgbm = create_model('lightgbm')

In [ ]:
print(lightgbm)

In [ ]:
tuned_lightgbm = tune_model(lightgbm,optimize = 'Accuracy')

In [ ]:
plot_model(tuned_lightgbm, plot = 'auc')

In [ ]:
plot_model(tuned_lightgbm, plot = 'pr')

In [ ]:
plot_model(tuned_lightgbm, plot='feature')

In [ ]:
plot_model(tuned_lightgbm, plot = 'confusion_matrix')

In [ ]:
evaluate_model(tuned_lightgbm)

In [ ]:
predict_model(lightgbm)

In [ ]:
unseen_predictions = predict_model(tuned_lightgbm, data=data_unseen)
unseen_predictions.head()

In [ ]:
print("Confidence Score :   {}".format(round(unseen_predictions.Score.mean(),2)))#Confidence Score